*This project involves performing Principal Component Analysis (PCA) on a dataset of economic indicators to reduce the dimensionality of the data and identify the most important factors driving the variability in the data. The project then uses linear regression to model the relationship between the principal components and a target variable, the S&P 500 stock market index. The project evaluates the performance of the regression model using cross-validation and reports the mean R-squared score. The project uses Python, pandas, NumPy, scikit-learn, and regular expressions.*

# Import Data and Manipulations

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import re

In [2]:
# Load data
df_2010 = pd.read_excel("C:/jibrile/Downloads/current_copy.xlsx")
df_2010.rename(columns={'sasdate': 'Date'}, inplace=True)
df_2010 = df_2010.set_index('Date')
df_2010 = df_2010.iloc[1:, :]
df_2010.index = pd.to_datetime(df_2010.index, format="%Y%m%d")
df_2010_columns = list(df_2010.columns.values)

# Standardize data
for col in df_2010_columns[1:]:
    df_2010[col] = (df_2010[col] - df_2010[col].mean()) / df_2010[col].std()

# Remove unnecessary columns
columns_to_remove = ["ACOGNO", "ANDENOx_transformed", "TWEXAFEGSMTHx"]
df_2010.drop(columns = columns_to_remove, inplace = True)
df_2010 = df_2010.iloc[:617]

# Prepare data for regression
X = df_2010.drop('S&P 500_transformed', axis=1).fillna(0)
y = df_2010['S&P 500_transformed'].fillna(0)

# Forecast Returns using principal components X.

Forecast returns using 1 to 5 principal components X. Construct a table with a column
for each model specification, and rows that report predictive coefficients, t-stats, and
R2
. Interpret your findings

- Functions: 

In [3]:
# Define a function to perform PCA
def perform_pca(X):
    # 1. compute covariance matrix
    C = X.cov()

    # 2. compute eigenvector and eigenvalues of covariance matrix
    eigenvalues, eigenvector = np.linalg.eig(C)

    # 3. sort eigenvalues and eigenvectors in descending order, reorder the corresponding eigenvectors to match.
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvector[:, sorted_indices]

    return sorted_eigenvectors, sorted_eigenvalues

# Define a function to project data onto principal components and perform regression
def project_and_regress(X, y, num_pcs):
    # Perform PCA
    eigenvectors, eigenvalues = perform_pca(X)

    # Extract the real part of the eigenvectors
    eigenvectors = eigenvectors.real

    # Initialize results list
    results = []

    # Perform regression for each number of PCs
    for i in range(1, num_pcs+1):
        # Select principal components
        selected_eigenvectors = eigenvectors[:, :i]

        # Project original data onto the selected principal components to get new features
        X_projected = np.dot(X, selected_eigenvectors)

        # Perform regression analysis
        coefficients, t_stats, r_squared = regression_analysis(X_projected, y)

        # Extract real part of numbers
        coefficients = coefficients.real
        t_stats = t_stats.real
        r_squared = r_squared.real

        # Collect results
        for j in range(i):
            results.append({
                'Model': f'Model with {i} PC(s)',
                'Coefficient': coefficients[j+1],  # Ignoring the intercept
                't-Statistic': t_stats[j+1],  # Ignoring the t-stat of intercept
                'R-squared': r_squared
            })

    # Create DataFrame from results
    results_df = pd.DataFrame(results)

    # Return results DataFrame along with eigenvectors
    return results_df, eigenvectors

# Define a function to evaluate regression model
def evaluate_regression_model(X, y, regression_model):
    # Perform cross-validation
    scores = cross_val_score(regression_model, X, y, cv=5, scoring='r2')

    # Calculate mean score
    mean_score = np.mean(scores)

    return mean_score

# Define a function to print the contribution of each original variable to each principal component
def print_pc_contributions(eigenvectors, original_variable_names, num_pcs):
    for i in range(num_pcs):
        print(f"Principal Component {i+1}:")
        contributions = eigenvectors[:, i]
        sorted_indices = np.argsort(np.abs(contributions))[::-1]
        for j in sorted_indices:
            print(f"   {original_variable_names[j]}: {contributions[j]}")
        print("\n")

def regression_analysis(X, y):
    """Perform linear regression and return coefficients, t-stats, and R-squared."""
    # Adding intercept term
    X = np.hstack([np.ones((X.shape[0], 1)), X])
    # Calculating coefficients
    coefficients = np.linalg.inv(X.T @ X) @ X.T @ y
    # Predictions
    y_pred = X @ coefficients
    # Residuals
    residuals = y - y_pred
    # Mean Squared Error
    mse = np.sum(residuals**2) / (X.shape[0] - X.shape[1])
    # Standard error
    se = np.sqrt(np.diagonal(mse * np.linalg.inv(X.T @ X)))
    # t-statistics
    t_stats = coefficients / se
    # R-squared
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)

    return coefficients, t_stats, r_squared

- Perform PCA and Regression

In [4]:
results_df, eigenvectors = project_and_regress(X, y, 5)

- Evaluate Model

In [5]:
# Get the best model based on the number of PCs with the highest R-squared value
best_model_row = results_df.loc[results_df['R-squared'].idxmax()]
best_model_name = best_model_row['Model']

# Extract the number of PCs from the best model name
match = re.search(r'(\d+) PC\(s\)', best_model_name)
if match:
    num_pcs = int(match.group(1))
else:
    raise ValueError(f"Could not extract the number of PCs from the best model name: {best_model_name}")

# Get the projected data for the best model
X_projected = np.dot(X, eigenvectors[:, :num_pcs])

# Perform regression on the projected data for the best model
regression_model = LinearRegression()
regression_model.fit(X_projected, y)

# Evaluate the regression model for the best model
mean_score = evaluate_regression_model(X_projected, y, regression_model)
print(f"Mean R-squared score for {best_model_name}: {mean_score}")

Mean R-squared score for Model with 5 PC(s): -0.5822309519510698


- Print Contributions to PCs

In [6]:
print_pc_contributions(eigenvectors, X.columns, 5)

Principal Component 1:
   IPNCONGD: -0.15976902093600936
   IPNMAT: -0.1579007813308438
   CUSR0000SAD: -0.15491099421338458
   IPCONGD: -0.15448769609532895
   USWTRADE: -0.15201165740062156
   CPIAPPSL: -0.15133464543591002
   USTRADE: -0.1510038265687307
   IPFPNSS: -0.14840597616812454
   USGOVT: -0.14765477131622212
   USFIRE: -0.146315337059657
   IPFINAL: -0.14610482862698185
   USTPU: -0.14549874949960329
   DDURRG3M086SBEA: -0.14335338559515493
   IPMANSICS: -0.14130208785796217
   IPB51222S: -0.14036471478832302
   CLF16OV: -0.13991162761688383
   CE16OV: -0.13872992211641383
   PAYEMS: -0.1382233703030321
   USCONS: -0.13750105973205579
   IPDCONGD: -0.13703463454818862
   INDPRO: -0.13589666035445694
   SRVPRD: -0.134952505372898
   CUSR0000SAC: -0.13156111672722734
   IPBUSEQ: -0.12642459546691548
   DNDGRG3M086SBEA: -0.1263627432837091
   PCEPI: -0.12556813375069936
   CUSR0000SA0L2: -0.12310265317582861
   CUSR0000SA0L5: -0.12184524379711158
   CPITRNSL: -0.1216668757585

- Table and PC Contributions

In [7]:
results_df

,Model,Coefficient,t-Statistic,R-squared
0,Model with 1 PC(s),0.001237,0.190650,0.000059
1,Model with 2 PC(s),0.001237,0.191150,0.006902
2,Model with 2 PC(s),0.023691,2.056861,0.006902
3,Model with 3 PC(s),0.001237,0.191424,0.011354
4,Model with 3 PC(s),0.023691,2.059807,0.011354
5,Model with 3 PC(s),-0.022458,-1.661389,0.011354
6,Model with 4 PC(s),0.001237,0.197103,0.069024
7,Model with 4 PC(s),0.023691,2.120914,0.069024
8,Model with 4 PC(s),-0.022458,-1.710677,0.069024
9,Model with 4 PC(s),0.100067,6.157187,0.069024


---

interpretation: 
- Model with 1 PC: This model explains a very small portion of the variance in the dependent variable (R-squared is close to 0). The coefficient is positive but not statistically significant (t-statistic is much less than 2). Adding more principal components is necessary to improve the model's explanatory power.


- Model with 2 PCs: Adding the second PC increases the R-squared value slightly, indicating a marginally better fit. The second PC has a positive coefficient with a t-statistic of 2.056, suggesting weak statistical significance at the 5% level. This suggests that the second PC has some explanatory power, but its effect is not strong.


- Models with 3 to 5 PCs: As more PCs are added, the R-squared values increase, indicating better model fit. However, not all added PCs are statistically significant. For instance, in the model with 5 PCs, the last principal component (PC5) has a large negative coefficient and a high t-statistic, indicating a significant inverse relationship with the dependent variable and strong explanatory power.
